a Sanford Health dataset that represents the ED (Emergency Department) volumes presenting to 4 Sanford Health medical centers 1/1/18 through 8/31/2025.
The dataset provides the facility, date, hour, reason, number of total encounters, number of those encounters admitted to a floor.

Forecast the future daily volumes (number of total encounters and the number of encounters admitted to a floor) for the month of September and October CY2025 (where CY means calendar year), by day.

In [43]:
# pip install polars

In [44]:
import polars as pl

df = pl.read_csv("DSU-Dataset.csv")

In [45]:
df.head()

Site,Date,Hour,REASON_VISIT_NAME,ED Enc,ED Enc Admitted
str,str,f64,str,i64,i64
"""A""","""2018-01-01""",0.0,"""BREATHING PROBLEM""",1,1
"""A""","""2018-01-01""",0.0,"""BURN""",1,0
"""A""","""2018-01-01""",0.0,"""SHAKING""",1,1
"""A""","""2018-01-01""",0.0,"""VOMITING""",1,0
"""A""","""2018-01-01""",1.0,"""FROSTBITE""",1,0


In [46]:
col = df["Date"].str.to_date()
df = df.with_columns(col)

In [52]:
df.describe()

statistic,Site,Date,Hour,REASON_VISIT_NAME,ED Enc,ED Enc Admitted
str,str,str,f64,str,f64,f64
"""count""","""1174310""","""1174310""",1.17431e6,"""1168105""",1.17431e6,1.17431e6
"""null_count""","""0""","""0""",0.0,"""6205""",0.0,0.0
"""mean""",null,"""2021-12-05 23:13:59.314320""",13.549324,null,1.084646,0.291522
"""std""",null,null,6.142173,null,0.322814,0.47727
"""min""","""A""","""2018-01-01""",0.0,"""ABDOMINAL AORTIC ANEURYSM""",1.0,0.0
"""25%""",null,"""2019-12-23""",10.0,null,1.0,0.0
"""50%""",null,"""2021-12-30""",14.0,null,1.0,0.0
"""75%""",null,"""2023-11-11""",19.0,null,1.0,1.0
"""max""","""D""","""2025-08-31""",23.0,"""WRIST PAIN""",8.0,5.0


The forecasted volumes are not to be broken down by reason of visit.

In [48]:
import numpy as np

columns = df.columns

for col in columns:
    print(f"Column {col} has {df[col].is_null().sum()} missing values.")

print(f"ED Enc has a min of {df['ED Enc'].min()} and a max of {df['ED Enc'].max()} at a particular hour for a certain reason")
print(f"ED Enc Admitted has a min of {df['ED Enc Admitted'].min()} and a max of {df['ED Enc Admitted'].max()} at a particular hour for a certain reason")
print(f"There are sites: {', '.join([site for site in df['Site'].unique()])}")

Column Site has 0 missing values.
Column Date has 0 missing values.
Column Hour has 0 missing values.
Column REASON_VISIT_NAME has 6205 missing values.
Column ED Enc has 0 missing values.
Column ED Enc Admitted has 0 missing values.
ED Enc has a min of 1 and a max of 8 at a particular hour for a certain reason
ED Enc Admitted has a min of 0 and a max of 5 at a particular hour for a certain reason
There are sites: C, D, A, B


Combine encounters by months, then facilities. See trends over months

In [64]:
start_date = df["Date"].min()
end_date = df["Date"].max()
date_range = pl.date_range(start=start_date, end=end_date, interval="1mo",  eager=True)

monthly_data = df.clear().drop(["Hour", "REASON_VISIT_NAME"])

for single_date in date_range.to_list():
    # get the data for the specific month
    filter_monthly = df.filter(
        (pl.col("Date").dt.month() == single_date.month) & (pl.col("Date").dt.year() == single_date.year)
    )

    for site in ['A', 'B', 'C', 'D']:
        monthly_data = pl.concat([monthly_data, 
                                  pl.DataFrame({
                                        "Site": site,
                                        "Date": single_date,
                                        "ED Enc": filter_monthly.filter(pl.col("Site") == site).select(pl.col("ED Enc")).sum(),
                                        "ED Enc Admitted": filter_monthly.filter(pl.col("Site") == site).select(pl.col("ED Enc Admitted")).sum()
                                  })
        ])
    print(monthly_data)


shape: (4, 4)
┌──────┬────────────┬────────┬─────────────────┐
│ Site ┆ Date       ┆ ED Enc ┆ ED Enc Admitted │
│ ---  ┆ ---        ┆ ---    ┆ ---             │
│ str  ┆ date       ┆ i64    ┆ i64             │
╞══════╪════════════╪════════╪═════════════════╡
│ A    ┆ 2018-01-01 ┆ 3888   ┆ 1182            │
│ B    ┆ 2018-01-01 ┆ 4944   ┆ 1150            │
│ C    ┆ 2018-01-01 ┆ 2621   ┆ 830             │
│ D    ┆ 2018-01-01 ┆ 2204   ┆ 358             │
└──────┴────────────┴────────┴─────────────────┘
shape: (8, 4)
┌──────┬────────────┬────────┬─────────────────┐
│ Site ┆ Date       ┆ ED Enc ┆ ED Enc Admitted │
│ ---  ┆ ---        ┆ ---    ┆ ---             │
│ str  ┆ date       ┆ i64    ┆ i64             │
╞══════╪════════════╪════════╪═════════════════╡
│ A    ┆ 2018-01-01 ┆ 3888   ┆ 1182            │
│ B    ┆ 2018-01-01 ┆ 4944   ┆ 1150            │
│ C    ┆ 2018-01-01 ┆ 2621   ┆ 830             │
│ D    ┆ 2018-01-01 ┆ 2204   ┆ 358             │
│ A    ┆ 2018-02-01 ┆ 3558   ┆ 1004      

In [65]:
import seaborn as sns
import matplotlib.pyplot as plt

# make some funny graphs